In [1]:
#multi-classification
#multi-nomial, classification Y값의 범주가 3개 이상

import tensorflow as tf
import numpy as np
tf.random.set_seed(5)

In [12]:
arr = np.loadtxt('data-04-zoo.csv',delimiter=',',dtype=np.float32)
arr.shape

(101, 17)

In [13]:
x_train=arr[:70,:-1]
y_train=arr[:70,[-1]]
x_test=arr[70:,:-1]
y_test=arr[70:,[-1]]
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(70, 16) (70, 1) (31, 16) (31, 1)


In [14]:
#one-hot encoding
nb_classes = 7
Y_one_hot = tf.one_hot(y_train,nb_classes)
print(Y_one_hot) #(N, M, L) Rank=3
Y_one_hot = tf.reshape(Y_one_hot,[-1,nb_classes])
print(Y_one_hot) #(N, L) Rank=2

tf.Tensor(
[[[1. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 1.]]

 [[0. 0. 0. 0. 0. 0. 1.]]

 [[0. 0. 0. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 1. 0.]]

 [[0. 0. 0. 0. 1. 0. 0.]]

 [[0. 0. 0. 0. 1. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 1. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0.

In [15]:
W=tf.Variable(tf.random.normal([16,nb_classes]),name='weight')
b=tf.Variable(tf.random.normal([nb_classes]),name='bias')
print(W,b)

<tf.Variable 'weight:0' shape=(16, 7) dtype=float32, numpy=
array([[ 1.4665463e+00, -1.0248139e+00,  8.6743283e-01, -2.2424309e+00,
         8.9757584e-02, -1.1256858e-01, -1.6262239e-01],
       [-3.1064131e-04, -2.0654018e+00,  9.5980629e-02, -1.1647935e+00,
        -5.9716773e-01, -1.6130395e+00,  5.5149209e-01],
       [-2.1156819e+00,  1.2274351e+00, -2.1617559e-01, -2.1591003e+00,
         8.7623733e-01, -1.4944816e-01, -1.0972142e+00],
       [ 4.4351053e-01, -3.7323128e-02,  1.6468747e+00,  5.8475316e-01,
         1.9537842e+00,  4.8303631e-01, -6.5814060e-01],
       [ 5.3004169e-01,  2.1524872e-01, -2.7115622e-01,  2.0796478e+00,
        -5.1800251e-01, -1.4942013e+00,  2.1573405e+00],
       [-1.1611416e-01, -4.6572775e-01,  1.0849730e+00,  2.5872741e+00,
         2.5746119e+00,  1.0033166e+00, -1.6337352e+00],
       [ 1.0855117e+00,  1.0968823e+00, -2.1023096e-01,  3.5354766e-01,
        -2.3794951e-01, -1.7822469e+00, -2.5174743e-01],
       [ 4.3639755e-01, -6.7254215e-0

In [24]:
#학습시는 logits를 사용
#비용(cost)계산 시 softmax...logits를 사용하므로 2번 호출하지 않아도 됨
#학습이 종료되서 생성된 모델을 가지고 예측함수 적용
def logits(X):
    return tf.matmul(X,W)+b
    
def hy(X):
    return tf.nn.softmax(logits(X))

In [21]:
def cost_func():
    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits(x_train),
                                                     labels=Y_one_hot)
    cost = tf.reduce_mean(cost_i)
    return cost

In [22]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

In [23]:
print("=====Start=====")
for step in range(5000):
    opt.minimize(cost_func,var_list=[W,b])
    if step%1000 == 0:
        print('%d'%step,'cost:',cost_func().numpy())
print("===finished===")

=====Start=====
0 cost: 0.00030315804
1000 cost: 1.5360898e-06
2000 cost: 4.802425e-07
3000 cost: 2.1117062e-07
4000 cost: 1.1410027e-07
===finished===


In [32]:
Yt_one_hot = tf.one_hot(y_test,nb_classes)
Yt_one_hot = tf.reshape(Yt_one_hot,[-1,nb_classes])

def predict(X):
    return tf.argmax(hy(X),axis=1)

correct_predict=tf.equal(predict(x_test),tf.argmax(Yt_one_hot,1))
accur = tf.reduce_mean(tf.cast(correct_predict, dtype=tf.float32))
print('Accuracy:',accur.numpy())

Accuracy: 0.8064516


In [34]:
pred= predict(x_test).numpy()
for p,y in zip(pred, y_test.flatten()):
    print("[{}] Prediction: {} / Real Y: {}".format(p == int(y), p, int(y)))

[True] Prediction: 0 / Real Y: 0
[True] Prediction: 1 / Real Y: 1
[False] Prediction: 5 / Real Y: 6
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 0 / Real Y: 0
[False] Prediction: 3 / Real Y: 2
[True] Prediction: 6 / Real Y: 6
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 2 / Real Y: 2
[False] Prediction: 1 / Real Y: 6
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 6 / Real Y: 6
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 5 / Real Y: 5
[True] Prediction: 4 / Real Y: 4
[False] Prediction: 1 / Real Y: 2
[False] Prediction: 4 / Real Y: 2
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 5 / Real Y: 5
[True] Prediction: 0 / Real Y: 0
[False] Prediction: 1 / Real Y: 6
[Tru